In [ ]:
from src.utils import utils
import json
import os 
import pandas as pd 

In [7]:
questions_list = [
  "What are the main objectives and focus areas of the meeting?",
  "Who are the participants, and what are their roles?",
  "What are the latest updates on assigned tasks and significant developments?",
  "What previous action items are still pending, and what challenges were faced?",
  "What major topics need to be discussed, and what insights have been shared?",
  "What challenges are currently being faced, and what solutions have been proposed?",
  "What key decisions need to be made, and what are their implications?",
  "What are the next steps, assigned tasks, and deadlines?",
  "What follow-up actions and plans are required for the next meeting?"
]
len(questions_list)

9

# Based on qa_by_rag, and template, create the agenda

In [1]:
# load qa_by_rag
import json
def load_qa_from_file(file, root:str = "/datadrive/CuongHV/project/DATA/mm_agenda_generation_research_output/qa_by_rag"):
    with open(f'{root}/{file}', 'r') as f:
        data = json.load(f)
    lines = [f'Question:{q}\nAnswer:{a}' for q,a in data['qa'].items()]
    qa_text = '\n'.join(lines)
    return qa_text


In [2]:
qa_text = load_qa_from_file('ES2002a.json')
print(qa_text)

Question:What are the main objectives and focus areas of the meeting?
Answer:The main objectives and focus areas of the meeting can be summarized as follows:

1. **Finalization of Design Prototype**: The meeting aimed to finalize the design prototype of the product, ensuring that all team members were aligned on the final design elements.

2. **Review of Cost Constraints**: A significant focus was placed on reviewing the financial constraints that impacted the design, including discussions on which features could be retained or had to be removed due to budget limitations.

3. **Evaluation of Project Success**: The meeting included an evaluation of the overall success of the project, assessing whether the objectives set at the beginning were met and how the team performed throughout the project lifecycle.

4. **Clarification of Roles and Responsibilities**: There was a need to clarify the roles and responsibilities of team members, as some members expressed uncertainty about their speci

In [3]:
from src.utils.generate import Generation 
from src.utils import utils
file_config = utils.load_config("src/config/file_config.yml")
model_config = utils.load_config(file_config["llm_env"]["model_config_file"])
prompt_config = utils.load_config(file_config["llm_env"]["prompting_file"])
generate = Generation(prompt_config)

In [4]:
from src.utils.llm_models import get_llm_model
import os
from dotenv import load_dotenv
load_dotenv()
config = {
    'openai_api_key': os.getenv("OPENAI_API_KEY"),
    'llm_choice': 'OpenAI', # Ollama
    'model_choice': 'gpt-4o-mini', # gpt-4o
    'parameters': {
        'temperature': 0.2,
        'top_p': 0.95,
        'max_retries': 2,
    },
}
llm = get_llm_model(
    chatmodel=config['llm_choice'], 
    model_name=config['model_choice'], 
    param=config['parameters'], 
)

In [7]:
from langchain_core.tracers.context import tracing_v2_enabled

with tracing_v2_enabled(project_name="generate_rag_multi_input_agenda"):
    agenda = generate.generate_rag_multi_input_agenda(llm=llm, qa_text=qa_text)
agenda 

{'text': '**Meeting Agenda: Product Development Update**  \n\n1. **Introductions and Meeting Overview (5 minutes)**  \n  + **Overview of Meeting Purpose (2 minutes)**  \n    + Finalization of design prototype and alignment on key features.  \n    + Review of cost constraints and implications for design decisions.  \n  + **Team Introductions (3 minutes)**  \n    + Brief introductions of team members and their roles.  \n    + Overview of project objectives and current status.  \n\n2. **Project Updates (10 minutes)**  \n  + **Status Reports (5 minutes)**  \n    + Team members provide updates on their assigned tasks and significant developments.  \n    + Highlight any challenges faced and solutions proposed.  \n  + **Review of Previous Action Items (5 minutes)**  \n    + Discuss pending action items and clarify roles and responsibilities.  \n    + Address any outstanding issues from the last meeting.  \n\n3. **Discussion Topics (10 minutes)**  \n  + **Key Design Decisions (5 minutes)**  \n

In [6]:
print(agenda['text'])

**Meeting Agenda: Product Development Update**  

1. **Introductions and Meeting Purpose (5 minutes)**  
  + **Overview of Meeting Objectives (2 minutes)**  
    + Finalization of design prototype and alignment on design elements.  
    + Review of cost constraints and implications for product features.  
  + **Team Introductions (3 minutes)**  
    + Each member introduces themselves and their role in the project.  

2. **Project Updates (10 minutes)**  
  + **Status Reports on Assigned Tasks (5 minutes)**  
    + Team members provide updates on their assigned tasks and any significant developments.  
    + Highlight challenges faced and solutions proposed.  
  + **Review of Previous Action Items (5 minutes)**  
    + Discuss the status of pending action items and clarify roles and responsibilities.  
    + Address any challenges encountered since the last meeting.  

3. **Discussion Topics (10 minutes)**  
  + **Product Design and Functionality (5 minutes)**  
    + Discuss the final

# Create 'Rag' Agenda from 67 meetings

In [1]:
from src.utils.generate import Generation 
from src.utils import utils
from src.utils.llm_models import get_llm_model
import os
from dotenv import load_dotenv
load_dotenv()
file_config = utils.load_config("src/config/file_config.yml")
model_config = utils.load_config(file_config["llm_env"]["model_config_file"])
prompt_config = utils.load_config(file_config["llm_env"]["prompting_file"])
generate = Generation(prompt_config)
config = {
    'openai_api_key': os.getenv("OPENAI_API_KEY"),
    'llm_choice': 'OpenAI', # Ollama
    'model_choice': 'gpt-4o-mini', # gpt-4o
    'parameters': {
        'temperature': 0.2,
        'top_p': 0.95,
        'max_retries': 2,
    },
}
llm = get_llm_model(
    chatmodel=config['llm_choice'], 
    model_name=config['model_choice'], 
    param=config['parameters'], 
)

In [2]:
# load qa_by_rag
import json
def load_qa_from_file(file, root:str = "/datadrive/CuongHV/project/DATA/mm_agenda_generation_research_output/qa_by_rag"):
    with open(f'{root}/{file}', 'r') as f:
        data = json.load(f)
    lines = [f'Question:{q}\nAnswer:{a}' for q,a in data['qa'].items()]
    qa_text = '\n'.join(lines)
    return qa_text


In [4]:
from langchain_core.tracers.context import tracing_v2_enabled
from tqdm import tqdm
root = "/datadrive/CuongHV/project/DATA/mm_agenda_generation_research_output/qa_by_rag"
for file in tqdm(os.listdir(root)):
    category, description = utils.extract_category(file)
    qa_text = load_qa_from_file(file)
    with tracing_v2_enabled(project_name="generate_category_rag_multi_input_agenda"):
        agenda = generate.generate_category_rag_multi_input_agenda(
            llm=llm, 
            category= category,
            description=description,
            qa_text=qa_text,
        )
    data = {
        'qa_text': qa_text,
        'agenda': agenda['text'],
    }
    with open(f'/datadrive/CuongHV/project/DATA/mm_agenda_generation_research_output/generate_category_rag_multi_input_agenda/{file}', 'a', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


100%|██████████| 67/67 [10:30<00:00,  9.42s/it]
